In [ ]:
import pandas as pd

In [ ]:
liked_books = ["14", "1334340", "12170167"]

In [ ]:
!head book_id_map.csv

In [ ]:
csv_book_mapping = {}
with open("book_id_map.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [ ]:
len(csv_book_mapping)

In [ ]:
!wc -l goodreads_interactions.csv

In [ ]:
!ls -lh | grep goodreads_interactions

In [ ]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")

        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue

        book_id = csv_book_mapping[csv_id]

        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

In [ ]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")

        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [ ]:
recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [ ]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [ ]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [ ]:
books_titles.head()

In [ ]:
books_titles[books_titles["book_id"].isin(top_recs)]

In [ ]:
all_recs = recs["book_id"].value_counts()

In [ ]:
all_recs

In [ ]:
all_recs = all_recs.to_frame().reset_index()

In [ ]:
all_recs.columns = ["book_id", "book_count"]

In [ ]:
all_recs

In [ ]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [ ]:
all_recs

In [ ]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [ ]:
all_recs.sort_values("score", ascending=False).head(10)

In [ ]:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score", ascending=False)

In [ ]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})